## CFM 301 DA4
### Q5
#### Jeongseop Yi (Patrick), j22yi

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats

In [4]:
q5_df = pd.read_sas('ca.sas7bdat')

q5_df['date'] = pd.to_datetime(q5_df['date'])
q5_df['year'] = q5_df['date'].dt.year
q5_df['month'] = q5_df['date'].dt.month
q5_df['yyyymm'] = q5_df['year'].astype(str) + q5_df['month'].astype(str).str.zfill(2)

q5_df = q5_df[(q5_df['yyyymm'] >= '200001') & (q5_df['yyyymm'] <= '202111')]

ff4_df = pd.read_sas('ff4.sas7bdat')
ff4_df.columns = map(str.lower, ff4_df.columns)
ff4_df['dateff'] = pd.to_datetime(ff4_df['dateff'])
ff4_df['year'] = ff4_df['dateff'].dt.year
ff4_df['month'] = ff4_df['dateff'].dt.month
ff4_df['yyyymm'] = ff4_df['year'].astype(str) + ff4_df['month'].astype(str).str.zfill(2)
ff4_df = ff4_df[(ff4_df['yyyymm'] >= '200001') & (ff4_df['yyyymm'] <= '202111')]

In [5]:
factors = ['ivol_winsorized', 'beta_winsorized', 'lnSize_winsorized', 'bk2mkt_winsorized']

for factor in factors:
    q5_df = q5_df[q5_df[factor].notna()]

test_df = q5_df.groupby('yyyymm').apply(lambda x: sm.OLS(x['ret_t1'], sm.add_constant(x[factors])).fit().params).reset_index()
test_df.columns = ['yyyymm', 'const', 'ivol', 'beta', 'lnSize', 'bk2mkt']
(t, p) = stats.ttest_1samp(test_df['ivol'], 0)
ivol_df = test_df[['yyyymm', 'ivol']]
ivol_df_des = ivol_df.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
ivol_df_des.loc['t-stat', :] = t.round(2)
display(ivol_df_des.round(4).T)

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max,t-stat
ivol,263.0,0.4145,2.7934,-9.3001,-5.5936,-4.1283,-1.0434,0.4017,1.728,4.8139,8.2413,14.0774,2.41


In [6]:
with pd.ExcelWriter("../DA4_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    q5_df.to_excel(writer, sheet_name="q5", index=False)